In [1]:
import os, yaml, sys
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from numba import njit
import warnings
from scipy.stats import zscore
ENV = os.getenv("MY_ENV", "dev")
with open("../../config.yaml", "r") as f:
    config = yaml.safe_load(f)
paths = config[ENV]["paths"]
sys.path.append(paths["src_path"])

In [2]:
@njit(nopython=True)
def mean(x):
    n = x.shape[0]
    tot = 0
    for i in x:
        tot+=i
    m = tot/n
    return m

def arr_mean(x, axis=0):
    if axis == 0:
        rows, cols = x.shape
        for 


/Users/tizianocausin/Desktop/virtual_envs/temporal_context/lib/python3.12/site-packages/numba/core/decorators.py:248: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [3]:
n = 10
xx = np.random.randn(n)
st = time.time()
xx.mean()
end = time.time()
print(end-st)
st = time.time()
mean(xx)
end = time.time()
print(end-st)

0.00022983551025390625
0.1858203411102295


In [4]:
@njit(nopython=True)
def std(x):
    n = x.shape[0]
    m_x2 = mean(x**2) # E[x^2]
    m2_x = mean(x)**2 # E[x]^2
    s = math.sqrt(m_x2 - m2_x)
    return s

/Users/tizianocausin/Desktop/virtual_envs/temporal_context/lib/python3.12/site-packages/numba/core/decorators.py:248: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [5]:
n = 10
xx = np.random.randn(n)
st = time.time()
xx.std()
end = time.time()
print(end-st)
st = time.time()
std(xx)
end = time.time()
print(end-st)

8.7738037109375e-05
0.1584031581878662


In [6]:
m2_x = mean(xx)**2 # E[x]^2
m_x2 = mean(xx**2)
print(m2_x)
print(m_x2)

0.0277395478221452
0.8438171791031734


In [7]:
@njit(nopython=True)
def z_score(x):
    n = x.shape
    m = mean(x)
    s = std(x)
    x_z = np.zeros(n)
    for i, val in enumerate(x):
        val = (val - m)/s
        x_z[i] = val
    return x_z

/Users/tizianocausin/Desktop/virtual_envs/temporal_context/lib/python3.12/site-packages/numba/core/decorators.py:248: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [8]:
n = 10
xx = np.random.randn(n)
st = time.time()
z_sp =zscore(xx)
end = time.time()
print(end-st)
st = time.time()
z_me = z_score(xx)
end = time.time()
print(end-st)

0.0007259845733642578
0.12873601913452148


In [9]:
@njit
def dot_prod(x1, x2):
    n1 = x1.shape[0]
    n2 = x2.shape[0]
    if n1 != n2:
        raise ValueError("x1, x2 do not have the same shapes : x1.shape = {n1}, x2.shape = {n2}")
    tot = 0
    #return (x1 * x2).sum()
    
    for el1, el2 in zip(x1, x2):
        tot += el1*el2
    
    return tot
    

In [10]:
x1 = np.random.randn(10000)
x2 = np.random.randn(10000)
 
st = time.time()
np.dot(x1, x2)
end = time.time()
print(end-st)
st = time.time()
dot_prod(x1, x2)
end = time.time()
print(end-st)

0.0010647773742675781
0.06492900848388672


In [11]:
@njit
def correlation(x1, x2):
    n = x1.shape[0]
    z_x1 = z_score(x1)
    z_x2 = z_score(x2)
    r = dot_prod(z_x1, z_x2)/n
    return r

In [12]:
x1 = np.random.randn(10000)
x2 = np.random.randn(10000)
 
st = time.time()
r_np = np.corrcoef(x1, x2)
end = time.time()
print(end-st)
st = time.time()
r = correlation(x1, x2)
end = time.time()
print(end-st)
print(r_np, r)

0.001100778579711914
0.09151983261108398
[[1.         0.01420742]
 [0.01420742 1.        ]] 0.014207421117526289


In [13]:
cols1 = 13
print((cols1**2 - cols1)/2)

78.0


In [110]:
@njit
def compute_corr_mat(x1, x2 = None): # it doesn't scale well :(
    rows1, cols1 = x1.shape
    if x2 is None:
        counter = 0
        tot_size = (cols1**2 - cols1)//2 # the number of elements in the upper triangular off-diag is (n^2 - n)/2
        corr_mat = np.zeros(tot_size)
        for icol in range(cols1):
            el1 = x1[:, icol]
            for jcol in range(icol):
                el2 = x1[:, jcol]
                r = correlation(el1, el2) # next make it ok for other kernels too
                corr_mat[counter] = r
                counter += 1
            # for jcol in range(icol-1):
        # for icol in range(cols1):
    else: 
        rows2, cols2 = x2.shape
        counter = 0
        tot_size = cols1*cols2  # the number of elements in the upper triangular off-diag is (n^2 - n)/2
        corr_mat = np.zeros(tot_size)
        #for icol in range(cols1):
        for col1 in x1:
            #el1 = x1[:, icol]
            #for jcol in range(cols2):
            for col2 in x2:
                #el2 = x2[:, jcol]
                #r = correlation(el1, el2) # next make it ok for other kernels too
                r = correlation(col1, col2) # next make it ok for other kernels too
                corr_mat[counter] = r
                counter += 1
            # for jcol in range(cols2):
        # for icol in range(cols1):
    return corr_mat

In [99]:
@njit
def corr(x1):
    X = x1 - x1.mean(axis=0)
    cov = X.T @ X / (X.shape[0] - 1)
    stds = X.std(axis=0, ddof=1)
    corr_mat = cov / (stds[:, None] * stds[None, :])
    return corr_mat

In [100]:
x = np.random.randn(200, 200)
x2 = np.random.randn(200, 200)
st = time.time()
corr(x)
end = time.time()
print(end-st)
x = x.T
st = time.time()
np.corrcoef(x,x2)
end = time.time()
print(end-st)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
- Resolution failure for literal arguments:
No implementation of function Function(<function array_mean at 0x120e6f920>) found for signature:

 >>> array_mean(array(float64, 2d, C), axis=Literal[int](0))

There are 2 candidate implementations:
    - Of which 2 did not match due to:
    Overload in function 'array_mean': File: numba/np/arraymath.py: Line 425.
      With argument(s): '(array(float64, 2d, C), axis=int64)':
     Rejected as the implementation raised a specific error:
       TypingError: got an unexpected keyword argument 'axis'
  raised from /Users/tizianocausin/Desktop/virtual_envs/temporal_context/lib/python3.12/site-packages/numba/core/typing/templates.py:791

- Resolution failure for non-literal arguments:
No implementation of function Function(<function array_mean at 0x120e6f920>) found for signature:

 >>> array_mean(array(float64, 2d, C), axis=int64)

There are 2 candidate implementations:
   - 

In [117]:
x = np.random.randn(200, 200)
x2 = np.random.randn(200, 200)
st = time.time()
compute_corr_mat(x,x2)
end = time.time()
print(end-st)
x = x.T
st = time.time()
np.corrcoef(x,x2)
end = time.time()
print(end-st)

0.09276580810546875
0.0010039806365966797
